# McsPyDataTools IMU Tutorial<a id='Top'></a>

- <a href='#Gyrosocope Data'>Gyroscope Data</a>
- <a href='#Accelerometer Data'>Accelerometer Data</a>
- <a href='#6DoF-Estimation'>Combined 6-DoF Data</a>

Tutorial to show the handling of IMU data collected from an MCS Headstage wearing an *Inertial Measurement Unit*

Load module and the data file:

In [ ]:
import sys
sys.path.append(r"../McsPyDataTools")
sys.executable

In [ ]:
# These are the imports of the McsData module
import McsPy.McsData
import McsPy.functions_info
from McsPy import ureg, Q_

# matplotlib.pyplot will be used in these examples to generate the plots visualizing the data
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.widgets import Slider
# These adjustments only need to be made so that the plot gets displayed inside the notebook
%matplotlib inline
# %config InlineBackend.figure_formats = {'png', 'retina'}

# numpy is numpy ...
import numpy as np
import McsPy.functions_info as fi

Check the content of the data file folder:

In [ ]:
fi.print_dir_file_info(".\\TestData")

Check what's inside the file:

In [ ]:
acc_gyro_raw_data_file_path = ".\\TestData\\2017-10-11T13-39-47McsRecording_X981_AccGyro.h5"
fi.print_file_info(acc_gyro_raw_data_file_path)

Load the file in silent mode:

In [ ]:
McsPy.McsData.VERBOSE = False
raw_data = McsPy.McsData.RawData(acc_gyro_raw_data_file_path)

## Gyroscope Data<a id='Gyroscope Data'></a>

In [ ]:
gyro_channel = raw_data.recordings[0].analog_streams[4]
print('Channel IDs: %s' % gyro_channel.channel_infos.keys())

In [ ]:
gyro = np.transpose(gyro_channel.channel_data)
gyro.shape

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(gyro)
#plt.title('Signal for Wireless (Simulation) / Raw ADC-Values (%s)' % analog_stream_0.label)
plt.xlabel('Sample Index')
plt.ylabel('Gyroscope Value')
plt.grid()

plt.show()

Cutout invalid data parts:

In [ ]:
gyro = gyro[0:10000,0:3]
gyro.shape

In [ ]:
time = gyro_channel.get_channel_sample_timestamps(148,0,10000)
gyro_x = gyro_channel.get_channel_in_range(148,0,10000)
gyro_y = gyro_channel.get_channel_in_range(149,0,10000)
gyro_z = gyro_channel.get_channel_in_range(150,0,10000)
plt.figure(figsize=(20,12))
plt.plot(time[0], gyro_x[0])
plt.plot(time[0], gyro_y[0])
plt.plot(time[0], gyro_z[0])
plt.xlabel('Time (%s)' % time[1])
plt.ylabel('Angular Speed (%s)' % gyro_x[1])
plt.title('Gyroscope Data')
plt.show()

## Accelerometer Data<a id='Accelerometer Data'></a>

In [ ]:
acc_channel = raw_data.recordings[0].analog_streams[5]
print('Channel IDs: %s' % acc_channel.channel_infos.keys())

In [ ]:
acc = np.transpose(acc_channel.channel_data)
acc.shape

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(acc)
#plt.title('Signal for Wireless (Simulation) / Raw ADC-Values (%s)' % analog_stream_0.label)
plt.xlabel('Sample Index')
plt.ylabel('Accelerometer Value')
plt.grid()

plt.show()

Cutout invalid data parts:

In [ ]:
acc = acc[0:10000,0:3]
acc.shape

In [ ]:
time = acc_channel.get_channel_sample_timestamps(160,0,10000)
acc_x = acc_channel.get_channel_in_range(160,0,10000)
acc_y = acc_channel.get_channel_in_range(161,0,10000)
acc_z = acc_channel.get_channel_in_range(162,0,10000)
plt.figure(figsize=(20,12))
plt.plot(time[0], acc_x[0])
plt.plot(time[0], acc_y[0])
plt.plot(time[0], acc_z[0])
plt.xlabel('Time (%s)' % time[1])
plt.ylabel('Acceleration (%s)' % acc_x[1])
plt.title('Accelerometer Data')
plt.show()

## Combined 6-DoF Data<a id='6DoF-Estimation'></a>

In [ ]:
import skinematics as skin
from skinematics.imus import IMU_Base

from scipy import constants # for "g"

from mpl_toolkits.mplot3d import Axes3D


Load data and separate values from unit:

In [ ]:
time, time_unit = gyro_channel.get_channel_sample_timestamps(148,0,10000)
gyro_x, gyro_x_unit = gyro_channel.get_channel_in_range(148,0,10000)
gyro_y, gyro_y_unit = gyro_channel.get_channel_in_range(149,0,10000)
gyro_z, gyro_z_unit = gyro_channel.get_channel_in_range(150,0,10000)

time, time_unit = acc_channel.get_channel_sample_timestamps(160,0,10000)
acc_x, acc_x_unit = acc_channel.get_channel_in_range(160,0,10000)
acc_y, acc_y_unit = acc_channel.get_channel_in_range(161,0,10000)
acc_z, acc_z_unit = acc_channel.get_channel_in_range(162,0,10000)

Define a new class for our data - derived from *IMU_Base* class in *scikit-kinematics*:

In [ ]:
class McsIMU(IMU_Base):
    """Concrete class based on abstract base class IMU_Base """    
    
    def get_data(self, in_file, in_data):
        '''Get the sampling rate, as well as the recorded data,
        and assign them to the corresponding attributes of "self".
        
        Parameters
        ----------
        in_file : string
                Filename of the data-file
        in_data : 
                Sampling rate (has to be provided!!)
        
        Assigns
        -------
        - rate : rate
        - acc : acceleration
        - omega : angular_velocity
        '''
        
        # The sampling rate has to be provided externally
        rate = in_data['rate']
            
        # Get the data, and label them
        data.columns = ['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z', 'mag_x', 'mag_y', 'mag_z', 'taccgyr', 'tmag']
            
        # Set the conversion factors by hand, and apply them
        #conversions = {}
        #conversions['time'] = 1/1000000
        #conversions['acc'] = 9.81
        #conversions['gyr'] = np.pi/180   
        #data[:,:3] *= conversions['acc']
        #data[:,3:6] *= conversions['gyr']
        #data[:,6] *= conversions['time']
            
        returnValues = [rate]
        
        # Extract the columns that you want, by name
        #paramList=['acc', 'gyr', 'mag']
        #for param in paramList:
        #    Expression = param + '*'
        #    returnValues.append(data_interp.filter(regex=Expression).values)
        returnValues.append(in_data['acc'])
        returnValues.append(in_data['gyro'])
        self._set_info(*returnValues)

# Set the conversion factors by hand, and apply them
conversions = {}
conversions['time'] = 1/1000000
conversions['acc'] = constants.g
conversions['gyr'] = np.pi/180

acc = np.column_stack((acc_x, acc_y, acc_z)) * conversions['acc']
gyro = np.column_stack((gyro_x, gyro_y, gyro_z)) * conversions['gyr']
time_second = time * conversions['time']

acc_sub = acc[::5,:].copy()
gyro_sub = gyro[::5,:].copy()

initial_orientation = np.array([[1,0,0],
                                [0,1,0],
                                [0,0,1]])
in_data = {"rate" : 2000, "acc" : acc, "omega" : gyro, "mag": None}
in_data_subsampled = {"rate" : 400, "acc" : acc_sub, "omega" : gyro_sub, "mag": None}
#mcs_imu = McsIMU(in_file = None, R_init = initial_orientation, in_data = in_data)
mcs_imu = McsIMU(in_file = None, R_init = initial_orientation, in_data = in_data_subsampled)
# mcs_imu.get_data(None, {'rate': 2000, 'acc': acc, 'gyro': gyro})

def show_result(imu_data):
        fig, axs = plt.subplots(3,1)
        axs[0].plot(imu_data.omega)
        axs[0].set_ylabel('Omega')
        axs[0].set_title(imu_data.q_type)
        axs[1].plot(imu_data.acc)
        axs[1].set_ylabel('Acc')
        axs[2].plot(imu_data.quat[:,1:])
        axs[2].set_ylabel('Quat')
        plt.show()

show_result(mcs_imu)

<a href='#Top'>Back to index</a>

In [ ]:
mcs_imu.q_type = 'analytical'
mcs_imu.calc_position()
pos_data = mcs_imu.pos

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot(pos_data[:,0], pos_data[:,1], pos_data[:,2], label='estimated position')
ax.legend()

plt.show()